In [59]:
from geostructures import *
from geostructures.collections import FeatureCollection, Track
from datetime import datetime


# To create a FeatureCollection, simply pass it a list of geoshapes
fcol = FeatureCollection(
    [
        GeoCircle(Coordinate(-0.118092, 51.509865), 500, dt=datetime(2020, 1, 1, 9, 5)),
        GeoEllipse(Coordinate(-0.118072, 51.502365), 2000, 400, 80, dt=datetime(2020, 1, 1, 9, 5)),
        GeoCircle(Coordinate(-0.141092, 51.529865), 500, dt=datetime(2020, 1, 1, 9, 5)),
        GeoRing(Coordinate(-0.116092, 51.519865), inner_radius=800, outer_radius=1000, dt=datetime(2020, 1, 1, 10, 5)),
        GeoRing(Coordinate(-0.101092, 51.514865), inner_radius=300, outer_radius=500, angle_min=60, angle_max=190, dt=datetime(2020, 1, 2, 11, 5)),
    ]
)



In [60]:
track = Track(fcol.geoshapes)

In [61]:
track.end

datetime.datetime(2020, 1, 2, 11, 10, tzinfo=datetime.timezone.utc)

In [46]:
def time_slice_track(track: Track, hour_interval: int):
    """
    Slices a Track into several tracks that are partitioned by a specified length
    of time in hours  
    
    Args:
        track: the target Track

        hour_interval= the length in hours of the desired interval

    Returns:
        A list of tracks
    """
    start_time = track.start
    end_time = track.end
    timestamps = []
    sliced_tracks = []

    # get timestamp intervals
    date_x = start_time
    while date_x < end_time:
        date_x += timedelta(hours = hour_interval)
        timestamps.append(date_x)
    
    # make the timestamp list inclusive
    timestamps[-1:] = timestamps[-1:] + timedelta(minutes = 1) 
    
    for timestamp in timestamps:
        sliced_track = track[start_time: timestamp]
        start_time = timestamp
        sliced_tracks.append(sliced_track)
    
    return sliced_tracks

In [62]:
test = time_slice_track(track, 1)

In [63]:
test

[<Track with 3 shapes from 2020-01-01T09:05:00+00:00 - 2020-01-01T09:05:00+00:00>,
 <Track with 1 shapes from 2020-01-01T10:05:00+00:00 - 2020-01-01T10:05:00+00:00>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Track with 1 shapes from 2020-01-02T11:10:00+00:00 - 2020-01-02T11:10:00+00:00>]

In [16]:
from datetime import datetime, timedelta
date_a = datetime(2016, 8, 9, 1, 24, 30, 993352)
date_b = datetime(2016, 8, 9, 7, 24, 30, 993352)

delta = timedelta(hours=1)
five_min_timestamps = []

date_x = date_a
while date_x < date_b:
    date_x += delta
    five_min_timestamps.append(date_x)

print(five_min_timestamps)

[datetime.datetime(2016, 8, 9, 2, 24, 30, 993352), datetime.datetime(2016, 8, 9, 3, 24, 30, 993352), datetime.datetime(2016, 8, 9, 4, 24, 30, 993352), datetime.datetime(2016, 8, 9, 5, 24, 30, 993352), datetime.datetime(2016, 8, 9, 6, 24, 30, 993352), datetime.datetime(2016, 8, 9, 7, 24, 30, 993352)]
